In [1]:
# private dock midestuary
# Data points every 5 minuetes
# HDR is sea level

import pandas as pd
import scipy.io as sio
from scipy import interpolate
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re
import json
%matplotlib ipympl

def Array_To_DataFrame(matDF, varName) :
    npArray = np.array([])
    for i in range (matDF[varName].shape[0]) :
        npArray = np.concatenate((npArray, matDF[varName][i].flatten()))
    return npArray
def matFile_To_DF(matFile) :
    tempDF = sio.loadmat(matFile)
    matDataMidFiltered = {}
    for dictKey in tempDF.keys() :
        if isinstance(tempDF[dictKey], np.ndarray) :
            if tempDF[dictKey].shape[0] > 1 :
                matDataMidFiltered.update({dictKey : tempDF[dictKey]})

    matMidDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMidFiltered.items()})
    return matMidDF

def matFileCurrent_To_DF(matFile) :
    tempDF = sio.loadmat(matFile)
    matDataMidFiltered = {}
    for dictKey in tempDF.keys() :
        if isinstance(tempDF[dictKey], np.ndarray) :
            if tempDF[dictKey].shape[0] > 1 and tempDF[dictKey].shape[1] > 1 :
                for i in range(tempDF[dictKey].shape[0]) :
                    matDataMidFiltered.update({dictKey + '_z_' + str(i) : tempDF[dictKey][i]})
            elif dictKey == 'DN':
                matDataMidFiltered.update({dictKey : tempDF[dictKey]})

    matMidDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMidFiltered.items()})
    matMidZ = pd.DataFrame({'z' : tempDF['Z'].flatten()})
    return (matMidDF, matMidZ, tempDF)

Bottom1213DataDF = pd.read_csv("bottom_data_12-13.csv")
Bottom1314DataDF = pd.read_csv("bottom_data_13-14.csv")
Surface1213DataDF = pd.read_csv("surface_data_12-13.csv")
Surface1314DataDF = pd.read_csv("surface_data_13-14.csv")

# Temperature Gradients 12-13 
# Midestuary to Downriver
distanceMidDown = 2.64 # km
distanceMidUp = 2.62 #km
#12-13 Surface
tempGradMidDownSurface1213 = (Surface1213DataDF['midestuary'] - Surface1213DataDF['downriver']) / distanceMidDown
tempGradMidUpSurface1213 = (Surface1213DataDF['midestuary'] - Surface1213DataDF['upriver']) / distanceMidUp
#12-13 Bottom
tempGradMidDownBottom1213 = (Bottom1213DataDF['midestuary'] - Bottom1213DataDF['downriver']) / distanceMidDown
tempGradMidUpBottom1213 = (Bottom1213DataDF['midestuary'] - Bottom1213DataDF['upriver']) / distanceMidUp
#12-13 average
tempMidDownAvg1213 = (Surface1213DataDF['downriver'] + Bottom1213DataDF['downriver']) / 2
tempMidUpAvg1213 = (Surface1213DataDF['upriver'] + Bottom1213DataDF['upriver']) / 2
tempMidAvg1213 = (Surface1213DataDF['midestuary'] + Bottom1213DataDF['midestuary']) / 2
tempGradAvgDown1213 = (tempMidAvg1213 - tempMidDownAvg1213)/ distanceMidDown
tempGradAvgUp1213 = (tempMidAvg1213- tempMidUpAvg1213)/ distanceMidUp
